<a href="https://colab.research.google.com/github/maxrgnt/pythdc2-project2/blob/master/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Panel Data
import pandas as pd
# System folders
import os
from pathlib import Path
# Visualization
import plotly.graph_objs as go
import seaborn as sns
%matplotlib inline

In [2]:
url = 'https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/unemployment.csv'
bls = pd.read_csv(url)
print(bls.shape)
bls.sample(1)

(322, 5)


,State,Year,Abrv,UnemploymentRate,LaborRate
176,New Mexico,2008,NM,0.046694,0.629993


In [3]:
url = 'https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/pctChangeGDP.csv'
bea = pd.read_csv(url)
print(bea.shape)
bea.sample(1)

(308, 4)


,State,Abrv,Year,Value
140,Alaska,AL,2007,10.2


In [4]:
# read in the data
url='https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/borderCrossing.csv'
bts = pd.read_csv(url)
print(bts.shape)
bts.sample(1)

(114588, 8)


,Abrv,State,Longitude,Latitude,Border,Year,Measure,Value
59270,ME,Maine,-67.79,46.77,US-Canada Border,2006,Pedestrians,2


In [5]:
def checkPctChange(df):
  print(df[(df['Abrv']=='TX') & (df['Year']==2014)])
  print(df[(df['Abrv']=='TX') & (df['Year']==2015)])
  print(df[(df['Abrv']=='TX') & (df['Year']==2016)])

In [6]:
def calcPctChange(df, col, pctChange_id):
  for y in list(df['Year'].unique()):
    states = list(df['Abrv'].unique())
    for s in states:
      if y != df['Year'].min():
        t0 = df.loc[(df['Year'] == y-1) & (df['Abrv'] == s), col].tolist()[0]
        t1 = df.loc[(df['Year'] == y) & (df['Abrv'] == s), col].tolist()[0]
        df.loc[(df['Year']==y) & (df['Abrv']==s), pctChange_id] = ((t1/t0)-1)*100
      else:
        df.loc[(df['Year']==y) & (df['Abrv']==s), pctChange_id] = 'N/A'
  # set drop index for first year where pctChange not calculated
  dropIndex = df.loc[df[pctChange_id]=='N/A'].index
  df.drop(dropIndex, inplace=True)

In [7]:
bts2 = bts.groupby(['Year','Abrv'])[['Value']].sum().reset_index()
bts2.sample(1)

,Year,Abrv,Value
135,2005,NM,2135676


In [8]:
calcPctChange(bts2, 'Value', 'border')
checkPctChange(bts2)

     Year Abrv     Value   border
263  2014   TX  80334766  4.71749
     Year Abrv     Value   border
277  2015   TX  82726100  2.97671
     Year Abrv     Value   border
291  2016   TX  85714503  3.61241


In [9]:
bea.rename(columns={'Value':'gdp'},inplace = True)
checkPctChange(bea)

     State Abrv  Year  gdp
249  Texas   TX  2014  4.2
     State Abrv  Year  gdp
263  Texas   TX  2015  0.2
     State Abrv  Year  gdp
277  Texas   TX  2016 -0.2


In [10]:
calcPctChange(bls, 'UnemploymentRate', 'unemp')
calcPctChange(bls, 'LaborRate', 'labor')
checkPctChange(bls)

     State  Year Abrv  UnemploymentRate  LaborRate    unemp     labor
263  Texas  2014   TX          0.053764    0.64867 -19.4988 -0.627148
     State  Year Abrv  UnemploymentRate  LaborRate    unemp    labor
277  Texas  2015   TX          0.046521   0.637932 -13.4719 -1.65538
     State  Year Abrv  UnemploymentRate  LaborRate    unemp      labor
291  Texas  2016   TX          0.048399   0.637718  4.03763 -0.0335446


In [11]:
df = bls[['Year','Abrv','unemp','labor']].merge(bea[['Year','Abrv','gdp']], how = 'left', left_on=['Year','Abrv'], right_on=['Year','Abrv'],sort=True)
df = df.merge(bts2[['Year','Abrv','border']], how = 'left', left_on=['Year','Abrv'], right_on=['Year','Abrv'],sort=True)

In [12]:
df['Year'] = df['Year'].astype(int)
for col in ['border','gdp','unemp','labor']:
  df[col] = df[col].astype(float)

In [13]:
df_melt = df.melt(['Year','Abrv'], var_name='PctChange', value_name='Values')

In [15]:
df_melt.sample(10)

,Year,Abrv,PctChange,Values
404,2005,VT,labor,0.140933
502,2012,VT,labor,-1.122415
760,2010,ME,gdp,2.300000
594,1998,MN,gdp,6.700000
745,2009,ID,gdp,-3.200000
1012,2007,ME,border,-24.574350
1053,2010,ID,border,17.061306
962,2003,NY,border,-17.304224
871,2018,ID,gdp,6.300000
872,2018,ME,gdp,4.300000


In [16]:
# save new data structure
df.to_csv(Path.joinpath(Path.cwd(),'data','master.csv'), index = False)

In [0]:
# sns.set(rc={'figure.figsize':(18,6)})
# sns.lineplot(x = 'Year', y = 'vals', hue = 'cols', ci=None, data = df_melt);
# sns.pairplot(df, vars=['border_pctChange','gdp_pctChange','unemp_pctChange'], kind='reg');

In [0]:
# mapbox_access_token = open("assets/mytoken.mapbox_token").read()

# fig = go.Figure(go.Scattermapbox(
#     lat=df['Latitude'],
#     lon=df['Longitude'],
#     mode='markers',
#     marker=go.scattermapbox.Marker(
#         size=20,
#         colorscale='Purples',
#         color=df['Value']
#     ),
#     text=df['Value']

# ))
# fig.update_layout(
#     autosize=True,
#     hovermode='closest',
#     mapbox=go.layout.Mapbox(
#         accesstoken=mapbox_access_token,
#         bearing=0,
#         center=go.layout.mapbox.Center(
#             lat=39.8283,
#             lon=-98.5795
#         ),
#         pitch=0,
#         zoom=3
#     ),
# )
# fig

In [0]:
# fig = go.Figure(data=go.Choropleth(
#     locations=df2['StateAbrv'], # Spatial coordinates
#     z = df2['Value'].astype(float), # Data to be color-coded
#     locationmode = 'USA-states', # set of locations match entries in `locations`
#     colorscale = 'Purples',
#     colorbar_title = "Pedestrians",
# ))

# fig.update_layout(
#     title_text = 'Migration',
#     geo_scope='usa', # limite map scope to USA
# )

# fig.show()